## ProjF5 - Final Model

Use this document as a template to provide the evaluation of your final model. You are welcome to go in as much depth as needed.

Make sure you keep the sections specified in this template, but you are welcome to add more cells with your code or explanation as needed.

In [2]:
import numpy as np
import matplotlib.pyplot as plt

### 1. Load and Prepare Data

This should illustrate your code for loading the dataset and the split into training, validation and testing. You can add steps like pre-processing if needed.

In [4]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the dataset
file_path = 'datasets/train_dataset.csv'  # Update this to the path of your dataset file
data = pd.read_csv(file_path)

# Display the first few rows of the dataset
print(data.head())

# Check for missing values
print(data.isnull().sum())

# Describe the dataset to understand its distribution
# print(data.describe())

#convert all columns to lowercase
data.columns = data.columns.str.lower()

#strip all symbols and replace with space, strip all tailing whitespaces and remaining white spaces replaced with underscores 
data.columns = data.columns.str.replace(r'[^a-zA-Z0-9]', ' ', regex=True).str.strip()
data.columns = data.columns.str.replace(' ', '_', regex=True)

# If you need to fill missing values, here's a simple way to do it (example)
# data.fillna(data.mean(), inplace=True)  # This fills missing values with the mean of each column

   age  height(cm)  weight(kg)  waist(cm)  eyesight(left)  eyesight(right)  \
0   35         170          85       97.0             0.9              0.9   
1   20         175         110      110.0             0.7              0.9   
2   45         155          65       86.0             0.9              0.9   
3   45         165          80       94.0             0.8              0.7   
4   20         165          60       81.0             1.5              0.1   

   hearing(left)  hearing(right)  systolic  relaxation  ...  HDL  LDL  \
0              1               1       118          78  ...   70  142   
1              1               1       119          79  ...   71  114   
2              1               1       110          80  ...   57  112   
3              1               1       158          88  ...   46   91   
4              1               1       109          64  ...   47   92   

   hemoglobin  Urine protein  serum creatinine   AST   ALT  Gtp  \
0        19.8            

In [6]:
data

,age,height_cm,weight_kg,waist_cm,eyesight_left,eyesight_right,hearing_left,hearing_right,systolic,relaxation,...,hdl,ldl,hemoglobin,urine_protein,serum_creatinine,ast,alt,gtp,dental_caries,smoking
0,35,170,85,97.0,0.9,0.9,1,1,118,78,...,70,142,19.8,1,1.0,61,115,125,1,1
1,20,175,110,110.0,0.7,0.9,1,1,119,79,...,71,114,15.9,1,1.1,19,25,30,1,0
2,45,155,65,86.0,0.9,0.9,1,1,110,80,...,57,112,13.7,3,0.6,1090,1400,276,0,0
3,45,165,80,94.0,0.8,0.7,1,1,158,88,...,46,91,16.9,1,0.9,32,36,36,0,0
4,20,165,60,81.0,1.5,0.1,1,1,109,64,...,47,92,14.9,1,1.2,26,28,15,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38979,40,165,60,80.0,0.4,0.6,1,1,107,60,...,61,72,12.3,1,0.5,18,18,21,1,0
38980,45,155,55,75.0,1.5,1.2,1,1,126,72,...,76,131,12.5,2,0.6,23,11,12,0,0
38981,40,170,105,124.0,0.6,0.5,1,1,141,85,...,48,138,17.1,1,0.8,24,23,35,1,1
38982,40,160,55,75.0,1.5,1.5,1,1,95,69,...,79,116,12.0,1,0.6,24,20,17,0,1


In [7]:
X, y = data.drop(['smoking'],axis=1),data['smoking']


In [8]:
def preprocess(df):
    df = pd.get_dummies(df,columns = ["hearing_left", "hearing_right","urine_protein","dental_caries"],dtype = 'int8')
    return df

In [9]:
X = preprocess(X)

In [10]:
# Normalize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # This will now be a NumPy array

# Ensure y is also a numpy array
y = y.values

### 2. Prepare your Final Model

Here you can have your code to either train (e.g., if you are building it from scratch) your model. These steps may require you to use other packages or python files. You can just call them here. You don't have to include them in your submission. Remember that we will be looking at the saved outputs in the notebooked and we will not run the entire notebook.

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier 
from tensorflow.keras.regularizers import l2
import tensorflow as tf

# Define a function to create the Keras model
def create_model(neurons=128, l2_rate=0.01):
    model = Sequential([
        Dense(neurons, activation='relu', input_dim=X.shape[1], kernel_regularizer=l2(l2_rate)),
        Dense(neurons, activation='relu', kernel_regularizer=l2(l2_rate)),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Wrap the model using KerasClassifier
model = KerasClassifier(build_fn=create_model, verbose=0)

# Define the grid search parameters
param_grid = {
    'model__neurons': [64, 128],
    'model__l2_rate': [0.001, 0.01, 0.02],
    'batch_size': [32, 64],
    'epochs': [20, 50]
}

# Setup cross-validation and grid search
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=kfold)

# Perform grid search with cross-validation
grid_result = grid.fit(X, y)  # No need to pre-scale; it will be handled internally by the grid search

# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#Best: 0.731351 using {'batch_size': 32, 'epochs': 50, 'model__l2_rate': 0.01, 'model__neurons': 64}


In [16]:
# Function to create the model, now just returning the model instance
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC, Accuracy

from sklearn.utils import class_weight
import numpy as np

# Define the K-fold cross validator
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calculate class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)

class_weights = dict(enumerate(class_weights))

def create_optimized_model(neurons = 64):
    model = Sequential([
        Input(shape = (X_scaled.shape[1],)),
        Dense(neurons, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(neurons, kernel_regularizer=l2(0.01)),
        Activation('relu'),

        Dense(neurons, kernel_regularizer=l2(0.01)),
        Activation('relu'),

        BatchNormalization(),
        Dropout(0.5),
        Dense(neurons, kernel_regularizer=l2(0.01)),
        Activation('relu'),

        Dense(neurons, kernel_regularizer=l2(0.01)),
        Activation('relu'),
        Dense(1, activation='sigmoid')
    ])
    optimizer = Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[AUC(), 'accuracy'])
    return model

# Assuming you have a setup for training and validation splits
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min'),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
]
# List to store each fold's accuracy
accuracies = []
aucs = []
# K-fold Cross Validation model evaluation
for train, test in kfold.split(X_scaled, y):
    model = create_optimized_model()
    # Fit data to model
    model.fit(X_scaled[train], y[train], epochs=20, batch_size=32, callbacks=callbacks, class_weight=class_weights, verbose=0)
    # Evaluate the model
    _, accuracy, auc = model.evaluate(X_scaled[test], y[test], verbose=0)
    accuracies.append(accuracy)
    aucs.append(auc)

# Print out the average and the standard deviation of the accuracies
average_accuracy = np.mean(accuracies)
std_deviation_accuracy = np.std(accuracies)
print(f'Average accuracy: {average_accuracy:.2f}, with standard deviation: {std_deviation_accuracy:.2f}')

#Average accuracy: 0.69, with standard deviation: 0.01

/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,auc_1,loss
  current = self.get_monitor_value(logs)
/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)
/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,auc_2,loss
  current = self.get_monitor_value(logs)
/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not availa

Average accuracy: 0.78, with standard deviation: 0.01


### SMOTE

In [13]:
from imblearn.over_sampling import SMOTE

In [14]:
# Function to create the model, now just returning the model instance
from tensorflow.keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, Input
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.utils import class_weight
import numpy as np

# Define the K-fold cross validator
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Calculate class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y), y=y)

class_weights = dict(enumerate(class_weights))

def create_optimized_model(neurons = 64):
    model = Sequential([
        Input(shape = (X_scaled.shape[1],)),
        Dense(neurons, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(neurons, kernel_regularizer=l2(0.01)),
        BatchNormalization(),
        Activation('relu'),
        Dropout(0.5),
        Dense(64, kernel_regularizer=l2(0.01)),
        Activation('relu'),
        Dense(1, activation='sigmoid')
    ])
    optimizer = Adam(learning_rate=0.01)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Assuming you have a setup for training and validation splits
callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min'),
    ModelCheckpoint('best_model.keras', monitor='val_loss', save_best_only=True, mode='min', verbose=1)
]
# List to store each fold's accuracy
accuracies = []

# K-fold Cross Validation model evaluation
for train, test in kfold.split(X_scaled, y):

    smote = SMOTE(random_state=42)
    X_train_scaled, y_train_scaled = smote.fit_resample(X_scaled[train], y[train])
    
    model = create_optimized_model()
    # Fit data to model
    model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=32, callbacks=callbacks, verbose=1)
    # Evaluate the model
    _, accuracy = model.evaluate(X_scaled[test], y[test], verbose=0)
    accuracies.append(accuracy)

# Print out the average and the standard deviation of the accuracies
average_accuracy = np.mean(accuracies)
std_deviation_accuracy = np.std(accuracies)
print(f'Average accuracy: {average_accuracy:.2f}, with standard deviation: {std_deviation_accuracy:.2f}')

#Average accuracy: 0.70, with standard deviation: 0.01

Epoch 1/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 796us/step - accuracy: 0.7067 - loss: 0.8503
Epoch 2/50
 186/1234 ━━━━━━━━━━━━━━━━━━━━ 0s 819us/step - accuracy: 0.7113 - loss: 0.6816

/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 803us/step - accuracy: 0.7191 - loss: 0.6722
Epoch 3/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 844us/step - accuracy: 0.7225 - loss: 0.6611
Epoch 4/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 781us/step - accuracy: 0.7189 - loss: 0.6600
Epoch 5/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 778us/step - accuracy: 0.7193 - loss: 0.6523
Epoch 6/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 783us/step - accuracy: 0.7183 - loss: 0.6482
Epoch 7/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 795us/step - accuracy: 0.7216 - loss: 0.6441
Epoch 8/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 846us/step - accuracy: 0.7189 - loss: 0.6397
Epoch 9/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 825us/step - accuracy: 0.7190 - loss: 0.6340
Epoch 10/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 829us/step - accuracy: 0.7222 - loss: 0.6299
Epoch 11/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 856us/step - accuracy: 0.7205 - loss: 0.6282
Epoch 12/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.7239 - loss: 0.6269
Epoch 13/50
1234/123

/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step - accuracy: 0.7235 - loss: 0.6648
Epoch 3/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 750us/step - accuracy: 0.7253 - loss: 0.6543
Epoch 4/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 765us/step - accuracy: 0.7225 - loss: 0.6529
Epoch 5/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 753us/step - accuracy: 0.7229 - loss: 0.6518
Epoch 6/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 752us/step - accuracy: 0.7217 - loss: 0.6434
Epoch 7/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 762us/step - accuracy: 0.7190 - loss: 0.6427
Epoch 8/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 763us/step - accuracy: 0.7265 - loss: 0.6332
Epoch 9/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 785us/step - accuracy: 0.7195 - loss: 0.6379
Epoch 10/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.7230 - loss: 0.6312
Epoch 11/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 868us/step - accuracy: 0.7207 - loss: 0.6317
Epoch 12/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 750us/step - accuracy: 0.7215 - loss: 0.6323
Epoch 13/50
1234/1

...

1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 852us/step - accuracy: 0.7175 - loss: 0.6750
Epoch 3/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 767us/step - accuracy: 0.7206 - loss: 0.6661
Epoch 4/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 811us/step - accuracy: 0.7207 - loss: 0.6543
Epoch 5/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 764us/step - accuracy: 0.7239 - loss: 0.6415
Epoch 6/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 896us/step - accuracy: 0.7227 - loss: 0.6398
Epoch 7/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 879us/step - accuracy: 0.7238 - loss: 0.6385
Epoch 8/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 790us/step - accuracy: 0.7223 - loss: 0.6388
Epoch 9/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 939us/step - accuracy: 0.7192 - loss: 0.6359
Epoch 10/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - accuracy: 0.7198 - loss: 0.6321
Epoch 11/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 806us/step - accuracy: 0.7224 - loss: 0.6245
Epoch 12/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 747us/step - accuracy: 0.7209 - loss: 0.6271
Epoch 13/50
1234/1

/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 820us/step - accuracy: 0.7248 - loss: 0.6665
Epoch 3/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 787us/step - accuracy: 0.7200 - loss: 0.6694
Epoch 4/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7190 - loss: 0.6586
Epoch 5/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7226 - loss: 0.6504
Epoch 6/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 807us/step - accuracy: 0.7233 - loss: 0.6470
Epoch 7/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.7194 - loss: 0.6447
Epoch 8/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 791us/step - accuracy: 0.7208 - loss: 0.6435
Epoch 9/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 786us/step - accuracy: 0.7234 - loss: 0.6376
Epoch 10/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 779us/step - accuracy: 0.7201 - loss: 0.6397
Epoch 11/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 774us/step - accuracy: 0.7251 - loss: 0.6384
Epoch 12/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7189 - loss: 0.6400
Epoch 13/50
1234/1234 ━━

/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/early_stopping.py:156: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/Users/opheliasin/smokerstatusNNDL/myenv/lib/python3.11/site-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 799us/step - accuracy: 0.7169 - loss: 0.6791
Epoch 3/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 818us/step - accuracy: 0.7227 - loss: 0.6661
Epoch 4/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 767us/step - accuracy: 0.7204 - loss: 0.6537
Epoch 5/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 765us/step - accuracy: 0.7226 - loss: 0.6519
Epoch 6/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7251 - loss: 0.6430
Epoch 7/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7234 - loss: 0.6318
Epoch 8/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 877us/step - accuracy: 0.7171 - loss: 0.6420
Epoch 9/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 808us/step - accuracy: 0.7149 - loss: 0.6370
Epoch 10/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 761us/step - accuracy: 0.7211 - loss: 0.6309
Epoch 11/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 780us/step - accuracy: 0.7193 - loss: 0.6315
Epoch 12/50
1234/1234 ━━━━━━━━━━━━━━━━━━━━ 1s 758us/step - accuracy: 0.7227 - loss: 0.6263
Epoch 13/50
1234/1234 

3. Model Performance
Make sure to include the following:

Performance on the training set
Performance on the test set
Provide some screenshots of your output (e.g., pictures, text output, or a histogram of predicted values in the case of tabular data). Any visualization of the predictions are welcome.